In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.3 MB/s eta 0:00:00


In [4]:
import os
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

def main():
    pipeline = AutoPipelineForInpainting.from_pretrained(
        "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
    )
    pipeline.enable_model_cpu_offload()
    # remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
    pipeline.enable_xformers_memory_efficient_attention()
    print("Diffusion Model ")

    image_path = "/content/drive/MyDrive/test_10/NewJeans 2nd EP _Get Up_.png"
    init_image = load_image(image_path)
    timestamp = image_path.split('/')[-1].split('.')[0]
    mask_image = load_image("/content/drive/MyDrive/mask/NewJeans 2nd EP _Get Up_.png")

    generator = torch.Generator("cuda").manual_seed(92) # always same generation

    # prompt = '''Fill the masked areas with background that matches the existing scene,
    # maintain color harmony and lighting consistency, seamless integration, natural blending,
    # remove all artifacts, preserve image quality, photorealistic completion, smooth transitions,
    # coherent composition, high resolution output'''

    # prompt = '''Fill masked area with background that perfectly matches surrounding colors and lighting,
    #     exact color temperature match, seamless edge blending, natural gradient continuation,
    #     preserve existing shadows and highlights, maintain image exposure and contrast,
    #     smooth color transitions, invisible patch boundaries, photorealistic integration,
    #     consistent ambient lighting, no color shift, perfect tonal harmony'''


    # prompt = '''Fill the masked areas with solid uniform color that exactly matches the nearest border pixels of each mask,
    # flat single color fill with no variations, smooth clean surface,
    # seamless edge blending from surrounding background, consistent lighting and color temperature,
    # invisible boundaries between mask and original image, perfect color continuation from edges'''

    # prompt = '''Fill the masked areas with solid uniform color that exactly matches the nearest border pixels of each mask,
    # flat single color fill with no variations, smooth clean surface.'''

    prompt = '''Fill the maksed areas with only skyblue color without any irregular and disformed patterns'''

    # prompt = "complete the scene, soft pastel gradient background, smooth color transition, dreamy pastel colors, seamless blend with surrounding image, no objects, no circles, clean background"
    negative_prompt = "fairy, color shift, pattern, color mismatch, lighting inconsistency, harsh edges, artifacts, unnatural transitions" # labels which were used at segmentation

    # higher strength: generate new image
    image = pipeline(prompt=prompt,
                     image=init_image, mask_image=mask_image, generator=generator,
                     strength=0.9, guidance_scale=15).images[0]

    # 출력 디렉토리 설정
    output_dir = "/content/drive/MyDrive/inpainted"
    os.makedirs(output_dir, exist_ok=True)

    # 개별 이미지들 저장
    inpainted_filename = f"{timestamp}.png"
    inpainted_path = os.path.join(output_dir, inpainted_filename)
    image.save(inpainted_path)
    print(f"Inpainted image saved: {inpainted_path}")

if __name__ == "__main__":
    main()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Diffusion Model 


  0%|          | 0/45 [00:00<?, ?it/s]

Inpainted image saved: /content/drive/MyDrive/inpainted/NewJeans 2nd EP _Get Up_.png
